## CNN 구현

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../mnist/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../mnist/data/t10k-images-idx3-ubyte.gz
Extracting ../mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1]) # [입력 데이터 개수, x축, y축, 특징의 개수]
Y = tf.placeholder(tf.float32, [None, 10]) # [입력 데이터 개수, 10개의 분류값]
keep_prob = tf.placeholder(tf.float32)

In [12]:
print(X)

Tensor("Placeholder_3:0", shape=(?, 28, 28, 1), dtype=float32)


In [13]:
# 오른쪽과 아래쪽으로 1칸씩 움직이는 32개의 커널을 가진 Convolution 계증 생성
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01)) # 3x3 크기 32개의 가중치
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME') # 'SAME' 는 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로 움직이는 옵션
L1 = tf.nn.relu(L1)

In [14]:
print(L1)

Tensor("Relu_3:0", shape=(?, 28, 28, 32), dtype=float32)


In [15]:
# pooling 계층
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # 2x2 크기의 pooling 계층

In [16]:
print(L1)

Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)


In [17]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01)) # 3x3 64개로 구성(32: W1의 커널개수)
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
print(L2)

Tensor("Relu_4:0", shape=(?, 14, 14, 64), dtype=float32)


In [18]:
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # 2x2 크기의 pooling 계층

In [19]:
print(L2)

Tensor("MaxPool_3:0", shape=(?, 7, 7, 64), dtype=float32)


In [21]:
# 10개의 분류층을 만들어내는 계층 구성(fully connected 계층)
# 7*7*64 숫자는 이전까지의 풀링 계층 크기가 7*7*64 임(print 문 참고)
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
print(L3)

Tensor("Relu_6:0", shape=(?, 256), dtype=float32)


In [22]:
L3 = tf.nn.dropout(L3, keep_prob)

In [23]:
print(L3)

Tensor("dropout_2/mul_1:0", shape=(?, 256), dtype=float32)


In [25]:
# L3의 출력값 256개를 받아 10개의 출력값을 만듦
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [26]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [27]:
# RMSPropOptimizer
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [29]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob:0.7})
        total_cost += cost_val
    print('Epoch : %04d' % (epoch +1), 'Avg cost = {:.3f}'.format(total_cost, total_batch))
print('최적화 완료')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : ', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob:1}))

Epoch : 0001 Avg cost = 505.487
Epoch : 0002 Avg cost = 52.928
Epoch : 0003 Avg cost = 32.433
Epoch : 0004 Avg cost = 26.118
Epoch : 0005 Avg cost = 20.581
Epoch : 0006 Avg cost = 17.574
Epoch : 0007 Avg cost = 15.418
Epoch : 0008 Avg cost = 13.139
Epoch : 0009 Avg cost = 11.624
Epoch : 0010 Avg cost = 10.127
Epoch : 0011 Avg cost = 9.339
Epoch : 0012 Avg cost = 8.961
Epoch : 0013 Avg cost = 8.052
Epoch : 0014 Avg cost = 7.430
Epoch : 0015 Avg cost = 7.484
최적화 완료
정확도 :  0.9913


## 고수준 API

In [33]:
# 컨볼루션 계층 변경
L1 = tf.layers.conv2d(X, 32, [3,3], activation=tf.nn.relu, padding='SAME')
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2], padding='SAME')

Instructions for updating:
Use keras.layers.MaxPooling2D instead.


In [34]:
# fully connected 계층 변경
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
